In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from pathlib import Path

In [31]:
data_path = Path("../data/tiny_shakespear.txt")

with data_path.open() as f:
    text = f.read()

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [32]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

"".join(chars), vocab_size

("\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz", 65)

In [33]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(e := encode("Hello There!"))
print(decode(e))

[20, 43, 50, 50, 53, 1, 32, 46, 43, 56, 43, 2]
Hello There!


In [34]:
# Train and test splits
data = torch.tensor(encode(text))
n = int(0.9*len(data)) 
train_data = data[:n]
val_data = data[n:]

train_data

tensor([18, 47, 56,  ..., 43, 56, 43])

In [35]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
epochs = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 200
# ------------

In [36]:
def get_batch(split):
    # random generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    # Average attention
    x, y = x.to(device), y.to(device)
    
    return x, y

In [37]:
x, y = get_batch("train")
x[0], y[0]

(tensor([47, 41, 43,  1, 39, 52, 42,  1]),
 tensor([41, 43,  1, 39, 52, 42,  1, 42]))

In [38]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        tokens = self.token_embedding_table(idx) # (B,T,C)
        logits = self.lm_head(tokens)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        model.eval()
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

In [39]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    
    return out

In [40]:
model = BigramLanguageModel().to(device)

In [41]:
model

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 200)
  (lm_head): Linear(in_features=200, out_features=65, bias=True)
)

In [42]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    # every once in a while evaluate the loss on train and val sets
    if epoch % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


step 0: train loss 4.2863, val loss 4.2934
step 300: train loss 2.5844, val loss 2.6284
step 600: train loss 2.5481, val loss 2.5816
step 900: train loss 2.5485, val loss 2.5572
step 1200: train loss 2.5182, val loss 2.5556
step 1500: train loss 2.4979, val loss 2.5355
step 1800: train loss 2.5217, val loss 2.5473
step 2100: train loss 2.5130, val loss 2.5386
step 2400: train loss 2.5036, val loss 2.5321
step 2700: train loss 2.4975, val loss 2.5208


In [43]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


RI:
!
I hive Heahave ngr em yoroul icisim, s o h I, qu isha shas:
Wh, th pro serenesachor CONERIVI V: ulcured; lde keard?
Y ul'se't!
SToveng lir y thone rse whimy co LON:
A 'e weamy ouendd hoffthy hay.
Y htsee POLERisng t mow be OURUS:




ARot
Mouras t.


F mas, ifalentam co s
WBI s Wouralir
F onoryeaang poknger h,

JOy s.
D t g:
Wh l he lonondalo SUMovonte limand:
Ay.

'Bum so ke y y cia me mousu sod gin, 'sore hat RENGr idert ble hidsine: mofilo, hie bes ds,

LBEO: lobustats m othe did I shem


In [44]:
torch.manual_seed(1337)

B, T, C = 4, 8, 2 # C = channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [45]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [46]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros_like(tril)
wei = wei.masked_fill(tril==0, -torch.inf)
wei = F.softmax(wei, dim=-1)

x_means = wei @ x
x_means[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [47]:
torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
a = a / a.sum(dim=-1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).to(torch.float)
c = a @ b

print('a=')
print(a)
print('--')
print('b=')
print(b)   
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
